In [1]:
from utils.dataset import get_dataloader
from utils.vulnerability_map import visualize_vulnerability_map, plot_triple_res
from utils.explainability_map import get_explainability_map
from detectors.models.WaveRep.detector import WaveRepDetector

ModuleNotFoundError: No module named 'support'

In [15]:
device = "cpu"

In [22]:
weights_dir = './weights/weights_AdversarialRobustnessCLIP'
detector = load_network('CORV_latent_r50', weights_dir).to(device)

is_resnet = type(detector).__name__ == "ResNet"

# Create dataloader AFTER loading the model so we can infer image size
images_dir: str = "./data/COCO_inpainted"
masks_dir: str = "./data/masks"
dataloader = get_dataloader(
    images_dir,
    masks_dir,
    batch_size=1,
    shuffle=True,
    num_workers=1,
    model=detector,
    transform_img=detector.preprocess if not is_resnet else None
    )

In [ ]:
detector

In [ ]:
for i, (image, mask, original) in enumerate(dataloader):
    print(f"BATCH {i}: image.shape={image.shape}, mask.shape={mask.shape}")
    explain_map, logits = get_explainability_map(image, detector, is_resnet=is_resnet, device=device)
    vis = visualize_vulnerability_map(explain_map, image)
    plot_triple_res(original, vis, mask)


In [22]:
# Create dataloader AFTER loading the model so we can infer image size
from utils.dataset import BatchedImageIterable
from torch.utils.data import DataLoader
images_dir: str = "./data/COCO_real"
masks_dir: str = "./data/masks"
dataset = BatchedImageIterable(
    images_dir,
    batch_size=1,
)
dataloader = DataLoader(dataset, batch_size=1, num_workers=1)  # dataset yields pre-batched items

In [17]:
import torch.nn.functional as F
import torch
from utils.utils import Metric
avg_estimate = Metric()
for i, (image, mask, orig) in enumerate(dataloader):
    with torch.no_grad():
        logits = detector(image.to(device))
        prob_ai = torch.sigmoid(logits[:, 1])
        for p in prob_ai:
            print(p)
        avg_estimate.update(prob_ai.mean().item(), n=prob_ai.numel())
        # res = detector(image.squeeze(0).to(device))[0][1]
        # print(res)
        # print("after sigmoid:")
        # print(F.sigmoid(res))
        # prob_AI_gen = F.sigmoid(res)
        # avg_estimate.update(prob_AI_gen.mean().item(), n=image.size(0))

print("Average estimated probability of AI-generated content:", avg_estimate.avg())



tensor(0.9729)
tensor(0.9812)
tensor(0.0741)
tensor(0.9804)
tensor(0.9473)
tensor(0.5510)
tensor(0.9593)
tensor(0.8772)
tensor(0.9951)
tensor(0.8999)
tensor(0.9843)
tensor(0.9981)
tensor(0.9480)
tensor(0.7102)
tensor(0.6353)
tensor(0.9992)
tensor(0.9924)
tensor(0.9972)
tensor(0.9794)
tensor(0.2528)
Average estimated probability of AI-generated content: 0.8367652732878923
